# Sentiment Analysis sulle Recensioni di Yelp

La Sentiment Analysis è il processo di identificazione dell'emozione espressa in un testo, positiva o negativa.

In questo notebook useremo Spark e la sua MLlib per costruire un modello di Sentiment Analysis usando il dataset messo a disposizione da Yelp, una famossisima applicazione che permette di recensire locali e attività commerciali.


# Inizializziamo Spark

In [17]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("sentiment_analysis").setMaster("local").set("spark.driver.memory", "5g")
sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

# Importiamo il dataset in un dataframe

In [18]:
yelp_df = sqlContext.read.json('./data/yelp_academic_dataset_review.json')

In [19]:
yelp_df.count()

6685900

In [20]:
yelp_df.columns

['business_id',
 'cool',
 'date',
 'funny',
 'review_id',
 'stars',
 'text',
 'useful',
 'user_id']

In [21]:
reviews_df = yelp_df.select(["text", "stars"])

In [23]:
subreviews_df = reviews_df.sample(False, 0.01, seed=0)

In [ ]:
subreviews_df.count()